In [8]:
#package loading 
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter
from dateutil.relativedelta import relativedelta
from scipy.stats import percentileofscore
import umap
from tqdm import tqdm
import cudf
import pickle
import sys

In [9]:
from utility import *
from preprocess import *

In [10]:
import warnings
warnings.filterwarnings("ignore")

*The tables below may have information of COVID-vaccine*

In [4]:
#death_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/death_cancer.pkl','rb'))
#diagnosis_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/diagnosis_cancer.pkl','rb'))
#encounter_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/encounter_cancer.pkl','rb'))
#lab_result_cm_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/lab_result_cm_cancer.pkl','rb'))
med_admin_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/med_admin_cancer.pkl','rb'))
obs_gen_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/obs_gen_cancer.pkl','rb'))
procedure_cancer=pickle.load( open('/data/datasets/changyuyin/2_COVIDwithCancer/procedure_cancer.pkl','rb'))
#demographics_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/demographics_cancer.pkl','rb'))
dispensing_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/dispensing_cancer.pkl','rb'))
immunization_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/immunization_cancer.pkl','rb'))
#lds_address_history_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/lds_address_history_cancer.pkl','rb'))
#obs_CLIN_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/obs_CLIN_cancer.pkl','rb'))
prescribing_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/prescribing_cancer.pkl','rb'))
#vital_cancer=pickle.load(open('/data/datasets/changyuyin/2_COVIDwithCancer/vital_cancer.pkl','rb'))

# Define codes representing COVID-19 vaccination

In [142]:
cpt_codes = tuple(set(['91300','0004A','91300','0001A','91300','0002A','91300',
                       '0003A','91301','0011A','91301','0012A','91301','0013A',
                       '91302','0021A','91302','0022A','91303','0034A','91303',
                       '0031A','91304','0044A','91304','0041A','91304','0042A',
                       '91305','0054A','91305','0051A','91305','0052A','91305',
                       '0053A','91306','0064A','91307','0074A','91307','0071A',
                       '91307','0072A','91307','0073A','91308','0081A','91308',
                       '0082A','91308','0083A','91309','0094A','91309','0091A',
                       '91309','0092A','91309','0093A','91310','0104A','91311',
                       '0111A','91311','0112A','91311','0113A','91312','0124A',
                       '91313','0134A','91314','0144A','91315','0154A','91316',
                       '0164A','91317','0173A','91303','91301','91306','91301',
                       '91309','91311','91313','91314','91316','91300','91305',
                       '91305','91307','91308','91312','91315','91317','91304',
                       '91310']))
print(len(cpt_codes))

58


In [143]:
cvx_codes = tuple(set(['212','207','207','207','221','228','229','230','208','217',
                   '217','218','219','300','301','302','211','225','213']))
print(len(cvx_codes))

16


In [144]:
rxnorm_codes = tuple(set(['2623378','2623382','2610328','2610319','2610347',
                      '2606078','2603740','2598700','2593847','2592364',
                      '2587906','2587899','2587898','2587313','2587312',
                      '2583743','2559909','2550903','2479835','2479154',
                      '2479153','2479150','2479149','2479159','2479158',
                      '2479156','2479155','2477902','2470234','2468235',
                      '2465246','2465248','2465253','2465255','2397257',
                      '2397251','2370236','2370234','2205474','2205472',
                      '2047247','2047241','2463118','2395502','2284960',
                      '2395504','2367758']))
print(len(rxnorm_codes))

47


In [145]:
ndc_codes = tuple(set(['5926710001','5926710001','5926710001','5926710001',
                   '8077727310','8077727310','8077727310','0310122210',
                   '0310122210','5967658005','5967658005','8063110001',
                   '8063110001','8063110001','5926710251','5926710251',
                   '5926710251','5926710251','8077727310','5926710551',
                   '5926710551','5926710551','5926710551','5926700781',
                   '5926700784','5926700784','8077727505','8077727505',
                   '8077727505','8077727505','4928161820','8077727905',
                   '8077727905','8077727905','5926703041','8077728205',
                   '8077728205','5926705651','8077728302','5926706091',
                   '59267100001','59267100001','59267100001','59267100001',
                   '80777027310','80777027310','80777027310','00310122210',
                   '00310122210','59676058005','59676058005','80631100001',
                   '80631100001','80631100001','59267102501','59267102501',
                   '59267102501','59267102501','80777027310','59267105501',
                   '59267105501','59267105501','59267105501','59267007801',
                   '59267007804','59267007804','80777027505','80777027505',
                   '80777027505','80777027505','49281061820','80777027905',
                   '80777027905','80777027905','59267030401','80777028205',
                   '80777028205','59267056501','80777028302','59267060901',
                   '5967658015','8077727399','8077727398','8077727399',
                   '8077727398','8077710099','8077727599','8077727999',
                   '8077728299','8077728399','5926710002','5926710003',
                   '5926710253','5926710254','0069202510','0069202525',
                   '5926710554','5926700784','5926703042','5926714042',
                   '5926705652','5926706092','8063110010','4928161820',
                   '0069100002','00069100003','8077710098','8077727799',
                   '8077728099','5967658005','8077727310','8077727315',
                   '8077727310','8077727315','8077710011','8077727505',
                   '8077727905','8077728205','8077728302','5926710001',
                   '5926710251','0069202501','5926710551','5926700781',
                   '5926703041','5926714041','5926705651','5926706091',
                   '8063110001','4928161878','0069100001','8077710015',
                   '8077727705','8077728005',]))
print(len(ndc_codes))

68


# Extract COVID-19 vaccination records

## from `immunization`

In [146]:
print(immunization_cancer.columns)
print(immunization_cancer.VX_CODE_TYPE.value_counts())

Index(['IMMUNIZATIONID', 'ID', 'ENCOUNTERID', 'PROCEDURESID', 'VX_PROVIDERID',
       'VX_RECORD_DATE', 'VX_ADMIN_DATE', 'VX_CODE_TYPE', 'VX_CODE',
       'VX_STATUS', 'VX_STATUS_REASON', 'VX_SOURCE', 'VX_DOSE', 'VX_DOSE_UNIT',
       'VX_ROUTE', 'VX_BODY_SITE', 'VX_MANUFACTURER', 'VX_LOT_NUM',
       'VX_EXP_DATE', 'SOURCE_masked'],
      dtype='object')
NI    86337
OT    35767
CX    26332
ND     2239
CH     1593
RX       10
Name: VX_CODE_TYPE, dtype: int64


In [147]:
immunization_cancer_vax = extract_COVIDvax_record(df=immunization_cancer,\
                             extract_from_col="VX_CODE",\
                             code_type_col="VX_CODE_TYPE",\
                             cpt_codes=cpt_codes,\
                             rxnorm_codes=rxnorm_codes,\
                             ndc_codes=ndc_codes,\
                             cvx_codes=cvx_codes)

Dimension before filtering:  (152278, 20)
Dimension after filtering:  (237, 20)


In [148]:
immunization_cancer_vax.isnull().sum()

IMMUNIZATIONID        0
ID                    0
ENCOUNTERID          20
PROCEDURESID        230
VX_PROVIDERID         0
VX_RECORD_DATE        0
VX_ADMIN_DATE         0
VX_CODE_TYPE          0
VX_CODE               0
VX_STATUS             0
VX_STATUS_REASON    107
VX_SOURCE             0
VX_DOSE               2
VX_DOSE_UNIT         19
VX_ROUTE              0
VX_BODY_SITE          0
VX_MANUFACTURER       0
VX_LOT_NUM            3
VX_EXP_DATE         107
SOURCE_masked         0
dtype: int64

## from `procedure`

In [149]:
print(procedure_cancer.columns)
print(procedure_cancer.PX_TYPE.value_counts())

Index(['PROCEDURESID', 'ID', 'ENCOUNTERID', 'ENC_TYPE', 'ADMIT_DATE',
       'PROVIDERID', 'PX_DATE', 'PX', 'PX_TYPE', 'PX_SOURCE', 'PPX',
       'SOURCE_masked'],
      dtype='object')
CH    23889820
RE     4181826
09      258530
10      228741
OT      172463
NI         879
Name: PX_TYPE, dtype: int64


In [150]:
procedure_cancer_vax = extract_COVIDvax_record(df=procedure_cancer,\
                             extract_from_col="PX",\
                             code_type_col="PX_TYPE",\
                             cpt_codes=cpt_codes,\
                             rxnorm_codes=rxnorm_codes,\
                             ndc_codes=ndc_codes,\
                             cvx_codes=cvx_codes)

Dimension before filtering:  (28732259, 12)
Dimension after filtering:  (6330, 12)


In [151]:
procedure_cancer_vax.isnull().sum()

PROCEDURESID        0
ID                  0
ENCOUNTERID         0
ENC_TYPE            0
ADMIT_DATE          0
PROVIDERID       1404
PX_DATE            25
VX_CODE             0
VX_CODE_TYPE        0
PX_SOURCE           0
PPX                 0
SOURCE_masked       0
dtype: int64

## from `prescribing`

In [152]:
print(prescribing_cancer.columns)

Index(['PRESCRIBINGID', 'ID', 'ENCOUNTERID', 'RX_PROVIDERID', 'RX_ORDER_DATE',
       'RX_ORDER_TIME', 'RX_START_DATE', 'RX_END_DATE', 'RX_DOSE_ORDERED',
       'RX_DOSE_ORDERED_UNIT', 'RX_QUANTITY', 'RX_DOSE_FORM', 'RX_REFILLS',
       'RX_DAYS_SUPPLY', 'RX_FREQUENCY', 'RX_PRN_FLAG', 'RX_ROUTE', 'RX_BASIS',
       'RXNORM_CUI', 'RX_SOURCE', 'RX_DISPENSE_AS_WRITTEN', 'SOURCE_masked'],
      dtype='object')


In [153]:
prescribing_cancer['CODE_TYPE']='RX'

In [154]:
prescribing_cancer['RXNORM_CUI'] = prescribing_cancer['RXNORM_CUI'].fillna('-9999999999').astype(int).astype(str)

In [155]:
prescribing_cancer_vax = extract_COVIDvax_record(df=prescribing_cancer,\
                             extract_from_col="RXNORM_CUI",\
                             code_type_col="CODE_TYPE",\
                             cpt_codes=cpt_codes,\
                             rxnorm_codes=rxnorm_codes,\
                             ndc_codes=ndc_codes,\
                             cvx_codes=cvx_codes)

Dimension before filtering:  (9278961, 23)
Dimension after filtering:  (3185, 23)


In [156]:
prescribing_cancer_vax.isnull().sum()

PRESCRIBINGID                0
ID                           0
ENCOUNTERID                  0
RX_PROVIDERID              329
RX_ORDER_DATE                0
RX_ORDER_TIME                0
RX_START_DATE              556
RX_END_DATE                570
RX_DOSE_ORDERED            342
RX_DOSE_ORDERED_UNIT         0
RX_QUANTITY               3173
RX_DOSE_FORM               555
RX_REFILLS                3171
RX_DAYS_SUPPLY            2748
RX_FREQUENCY                 0
RX_PRN_FLAG                  0
RX_ROUTE                    64
RX_BASIS                   554
VX_CODE                      0
RX_SOURCE                    0
RX_DISPENSE_AS_WRITTEN     554
SOURCE_masked                0
VX_CODE_TYPE                 0
dtype: int64

## from `dispensing`

In [157]:
print(dispensing_cancer.columns)

Index(['DISPENSINGID', 'ID', 'PRESCRIBINGID', 'DISPENSE_DATE', 'NDC',
       'DISPENSE_SOURCE', 'DISPENSE_SUP', 'DISPENSE_AMT', 'DISPENSE_DOSE_DISP',
       'DISPENSE_DOSE_DISP_UNIT', 'DISPENSE_ROUTE', 'SOURCE_masked'],
      dtype='object')


In [158]:
dispensing_cancer_vax.isnull().sum()

DISPENSINGID                  0
ID                            0
PRESCRIBINGID                 0
DISPENSE_DATE                 0
VX_CODE                       0
DISPENSE_SOURCE               0
DISPENSE_SUP                  0
DISPENSE_AMT                  0
DISPENSE_DOSE_DISP         1268
DISPENSE_DOSE_DISP_UNIT    1266
DISPENSE_ROUTE             1266
SOURCE_masked                 0
VX_CODE_TYPE                  0
dtype: int64

In [159]:
dispensing_cancer['CODE_TYPE']='ND'

In [160]:
dispensing_cancer['NDC'] = dispensing_cancer['NDC'].fillna('-9999999999').astype(str)

In [161]:
dispensing_cancer_vax = extract_COVIDvax_record(df=dispensing_cancer,\
                             extract_from_col="NDC",\
                             code_type_col="CODE_TYPE",\
                             cpt_codes=cpt_codes,\
                             rxnorm_codes=rxnorm_codes,\
                             ndc_codes=ndc_codes,\
                             cvx_codes=cvx_codes)

Dimension before filtering:  (4417243, 13)
Dimension after filtering:  (1300, 13)


In [162]:
dispensing_cancer_vax.isnull().sum()

DISPENSINGID                  0
ID                            0
PRESCRIBINGID                 0
DISPENSE_DATE                 0
VX_CODE                       0
DISPENSE_SOURCE               0
DISPENSE_SUP                  0
DISPENSE_AMT                  0
DISPENSE_DOSE_DISP         1268
DISPENSE_DOSE_DISP_UNIT    1266
DISPENSE_ROUTE             1266
SOURCE_masked                 0
VX_CODE_TYPE                  0
dtype: int64

# Assemble the data

In [164]:
print(len(immunization_cancer_vax['IMMUNIZATIONID']))
print(len(set(immunization_cancer_vax['IMMUNIZATIONID'])))
print(len(procedure_cancer_vax['PROCEDURESID']))
print(len(set(procedure_cancer_vax['PROCEDURESID'])))
print(len(prescribing_cancer_vax['PRESCRIBINGID']))
print(len(set(prescribing_cancer_vax['PRESCRIBINGID'])))
print(len(dispensing_cancer_vax['DISPENSINGID']))
print(len(set(dispensing_cancer_vax['DISPENSINGID'])))

237
237
6330
6286
3185
3077
1300
1300


In [ ]:
immunization_cancer_vax = immunization_cancer_vax.rename(columns={"VX_ADMIN_DATE":"VX_DATE"})
prescribing_cancer_vax = prescribing_cancer_vax.rename(columns={"RX_ORDER_DATE":"VX_DATE"})
procedure_cancer_vax = procedure_cancer_vax.rename(columns={"PX_DATE":"VX_DATE"})
dispensing_cancer_vax = dispensing_cancer_vax.rename(columns={"DISPENSE_DATE":"VX_DATE"})

In [ ]:
immunization_cancer_vax = immunization_cancer_vax.rename(columns={"IMMUNIZATIONID":"VX_ID"})
immunization_cancer_vax['VX_ID'] = "IMMUNIZATION"+immunization_cancer_vax['VX_ID']
immunization_cancer_vax['DOMAIN'] = "IMMUNIZATION"

prescribing_cancer_vax = prescribing_cancer_vax.rename(columns={"PRESCRIBINGID":"VX_ID"})
prescribing_cancer_vax['VX_ID'] = "PRESCRIBING"+prescribing_cancer_vax['VX_ID']
prescribing_cancer_vax['DOMAIN'] = "PRESCRIBING"

procedure_cancer_vax = procedure_cancer_vax.rename(columns={"PROCEDURESID":"VX_ID"})
procedure_cancer_vax['VX_ID'] = "PROCEDURE"+procedure_cancer_vax['VX_ID']
procedure_cancer_vax['DOMAIN'] = "PROCEDURE"

dispensing_cancer_vax = dispensing_cancer_vax.rename(columns={"DISPENSINGID":"VX_ID"})
dispensing_cancer_vax['VX_ID'] = "DISPENSING"+dispensing_cancer_vax['VX_ID']
dispensing_cancer_vax['DOMAIN'] = "DISPENSING"

In [ ]:
vax_cancer = pd.concat([immunization_cancer_vax[['ID','VX_ID','VX_CODE_TYPE','VX_CODE','VX_DATE','DOMAIN']],
                        prescribing_cancer_vax[['ID','VX_ID','VX_CODE_TYPE','VX_CODE','VX_DATE','DOMAIN']],
                        procedure_cancer_vax[['ID','VX_ID','VX_CODE_TYPE','VX_CODE','VX_DATE','DOMAIN']],
                        dispensing_cancer_vax[['ID','VX_ID','VX_CODE_TYPE','VX_CODE','VX_DATE','DOMAIN']]
                       ])

In [12]:
vax_cancer['VX_ID_RAW'] = vax_cancer['VX_ID']
vax_cancer['VX_ID'] = vax_cancer['ID']+vax_cancer['VX_CODE']+vax_cancer['VX_DATE']

In [13]:
print(vax_cancer.shape)

(11052, 7)


In [17]:
#drop duplicate information
vax_cancer = vax_cancer.drop_duplicates('VX_ID',ignore_index=True)

In [19]:
print(vax_cancer.shape)

(9273, 7)


In [20]:
pickle.dump(vax_cancer, open('/data/datasets/changyuyin/2_COVIDwithCancer/vax_cancer.pkl','wb'))

In [18]:
pickle.dump(immunization_cancer_vax, open('/data/datasets/changyuyin/2_COVIDwithCancer/immunization_cancer_vax.pkl','wb'))
pickle.dump(procedure_cancer_vax, open('/data/datasets/changyuyin/2_COVIDwithCancer/procedure_cancer_vax.pkl','wb'))
pickle.dump(prescribing_cancer_vax, open('/data/datasets/changyuyin/2_COVIDwithCancer/prescribing_cancer_vax.pkl','wb'))
pickle.dump(dispensing_cancer_vax, open('/data/datasets/changyuyin/2_COVIDwithCancer/dispensing_cancer_vax.pkl','wb'))